In [13]:
import os
import sys
import json
from os.path import join, basename, dirname

import time
import numpy as np
import pandas as pd

from feature_analyzer.index.agent import IndexAgent
from feature_analyzer.data_tools.embedding_container import EmbeddingContainer
from feature_analyzer.data_tools.result_container import ResultContainer
from feature_analyzer.evaluations.variance_evaluation import VarianceEvaluation

import seaborn as sns

In [5]:
root = '/home/kv_zhao/nist-e2e/feature-analyzer'
rm_embedding_container_path = join(root, 'examples/featobj_mergeV1_D40kv2_RM')
rmg_embedding_container_path = join(root, 'examples/featobj_mergeV1_D40kv2_RMG')
rm_result_path = join(root, 'outputs/var_mergeV1_D40kv2_RM')
rmg_result_path = join(root, 'outputs/var_mergeV1_D40kv2_RMG')

In [6]:
rm_container = EmbeddingContainer()
rm_results = ResultContainer()
rm_container.load(rm_embedding_container_path)
rm_results.load(rm_result_path)

Container:embedding_container created
Load embedding container from feat_obj format
/home/kv_zhao/nist-e2e/feature-analyzer/examples/featobj_mergeV1_D40kv2_RM/embeddings.npy is loaded
/home/kv_zhao/nist-e2e/feature-analyzer/examples/featobj_mergeV1_D40kv2_RM/label_ids.npy is loaded
/home/kv_zhao/nist-e2e/feature-analyzer/examples/featobj_mergeV1_D40kv2_RM/filename_strings.npy is loaded
/home/kv_zhao/nist-e2e/feature-analyzer/examples/featobj_mergeV1_D40kv2_RM/probabilities.npy is loaded
/home/kv_zhao/nist-e2e/feature-analyzer/examples/featobj_mergeV1_D40kv2_RM/label_names.npy is loaded
/home/kv_zhao/nist-e2e/feature-analyzer/examples/featobj_mergeV1_D40kv2_RM/landmarks.npy is loaded
/home/kv_zhao/nist-e2e/feature-analyzer/examples/featobj_mergeV1_D40kv2_RM/instance_ids.npy is loaded
container size: 10000 -> 134281
embedding size: 0 -> 1024
probability size: 0 -> 1
landmark size: 0 -> 10
Reset embedding_container
Index Table Created
Container initialized.
Load results and events from '/

In [7]:
rmg_container = EmbeddingContainer()
rmg_results = ResultContainer()
rmg_container.load(rmg_embedding_container_path)
rmg_results.load(rmg_result_path)

Container:embedding_container created
Load embedding container from feat_obj format
/home/kv_zhao/nist-e2e/feature-analyzer/examples/featobj_mergeV1_D40kv2_RMG/embeddings.npy is loaded
/home/kv_zhao/nist-e2e/feature-analyzer/examples/featobj_mergeV1_D40kv2_RMG/label_ids.npy is loaded
/home/kv_zhao/nist-e2e/feature-analyzer/examples/featobj_mergeV1_D40kv2_RMG/filename_strings.npy is loaded
/home/kv_zhao/nist-e2e/feature-analyzer/examples/featobj_mergeV1_D40kv2_RMG/probabilities.npy is loaded
/home/kv_zhao/nist-e2e/feature-analyzer/examples/featobj_mergeV1_D40kv2_RMG/label_names.npy is loaded
/home/kv_zhao/nist-e2e/feature-analyzer/examples/featobj_mergeV1_D40kv2_RMG/landmarks.npy is loaded
/home/kv_zhao/nist-e2e/feature-analyzer/examples/featobj_mergeV1_D40kv2_RMG/instance_ids.npy is loaded
container size: 10000 -> 134281
embedding size: 0 -> 1024
probability size: 0 -> 1
landmark size: 0 -> 10
Reset embedding_container
Index Table Created
Container initialized.
Load results and events 

In [8]:
results = rm_results
container = rm_container

In [10]:
events = results.events
num_events = len(events)

label_ids = list(map(int, events.label_id.unique()))
num_identities = len(label_ids)
num_instance_for_label_id = {
    label_id: len(container.get_instance_ids_by_label(label_id)) for label_id in label_ids
}

margin_events = events[events.margin >= 0]
no_margin_events = events[events.margin < 0]

num_typeI_events = len(margin_events)
num_typeII_events = len(no_margin_events)

print('[instance] TypeI : {}, TypeII: {}'.format(num_typeI_events / num_events, num_typeII_events / num_events))

tpyeI_identities = []
tpyeII_identities = []
for label_id in label_ids:
    num_margin_event = len(margin_events[margin_events.label_id == label_id])
    if num_margin_event == num_instance_for_label_id[label_id]:
        tpyeI_identities.append(label_id)
    else:
        # TODO: different levels
        tpyeII_identities.append(label_id)
num_tpyeI_identities = len(tpyeI_identities)
num_tpyeII_identities = len(tpyeII_identities)
print('[indentity] TypeI : {}, TypeII: {}'.format(num_tpyeI_identities/ num_identities, num_tpyeII_identities/ num_identities))

[instance] TypeI : 0.9058832290735775, TypeII: 0.0941167709264224
[indentity] TypeI : 0.8011719270209082, TypeII: 0.19882807297909175


In [14]:
no_margin_events

,extend_diversity,first_neg_index,first_neg_sim,instance_id,label_id,last_pos_index,last_pos_sim,margin,ret_ids,ret_label_ids,topk_purity
10,18.0,24.0,1.376589,10.0,2.0,26.0,1.356921,-0.019667,"[10, 27, 8, 11, 30, 14, 22, 28, 19, 33, 18, 12...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",0.961538
13,4.0,24.0,1.387033,13.0,2.0,27.0,1.355407,-0.031626,"[13, 29, 16, 20, 15, 25, 9, 19, 31, 8, 23, 32,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",0.923077
26,11.0,25.0,1.323361,26.0,2.0,36.0,1.283709,-0.039651,"[26, 33, 28, 17, 21, 19, 22, 23, 32, 16, 25, 2...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",0.961538
31,12.0,24.0,1.360128,31.0,2.0,46.0,1.283709,-0.076419,"[31, 29, 16, 23, 13, 19, 15, 11, 25, 20, 9, 21...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",0.923077
101,35.0,85.0,1.367904,101.0,6.0,93.0,1.331620,-0.036284,"[101, 113, 115, 144, 177, 109, 118, 149, 102, ...","[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...",0.988506
...,...,...,...,...,...,...,...,...,...,...,...
133923,19.0,36.0,1.413848,133924.0,7489.0,41.0,1.349249,-0.064598,"[133924, 133928, 133908, 133915, 133916, 13392...","[7489, 7489, 7489, 7489, 7489, 7489, 7489, 748...",0.972973
133924,22.0,35.0,1.427832,133925.0,7489.0,37.0,1.418373,-0.009459,"[133925, 133919, 133930, 133921, 133902, 13391...","[7489, 7489, 7489, 7489, 7489, 7489, 7489, 748...",0.972973
133959,20.0,43.0,1.359590,133960.0,7491.0,44.0,1.332114,-0.027477,"[133960, 133995, 133997, 133987, 133999, 13397...","[7491, 7491, 7491, 7491, 7491, 7491, 7491, 749...",0.977273
133976,21.0,43.0,1.331296,133977.0,7491.0,45.0,1.323932,-0.007364,"[133977, 133961, 133985, 133996, 133988, 13396...","[7491, 7491, 7491, 7491, 7491, 7491, 7491, 749...",0.977273


In [16]:
no_margin_events['extend_diversity_ratio'] = no_margin_events.apply(lambda x: x.extend_diversity / (len(x.ret_ids)/2), axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:
margin_events[margin_events.last_pos_sim < 1.5]

,extend_diversity,first_neg_index,first_neg_sim,instance_id,label_id,last_pos_index,last_pos_sim,margin,ret_ids,ret_label_ids,topk_purity
8,17.0,26.0,1.379969,8.0,2.0,25.0,1.405589,0.025620,"[8, 11, 10, 25, 18, 21, 19, 22, 28, 29, 33, 12...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",1.0
9,4.0,26.0,1.362035,9.0,2.0,25.0,1.480536,0.118501,"[9, 20, 23, 19, 24, 16, 21, 11, 25, 14, 29, 33...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",1.0
12,8.0,26.0,1.364743,12.0,2.0,25.0,1.463864,0.099121,"[12, 33, 19, 11, 23, 32, 21, 25, 18, 24, 14, 2...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",1.0
14,12.0,26.0,1.313590,14.0,2.0,25.0,1.429409,0.115818,"[14, 11, 21, 19, 23, 28, 33, 30, 18, 24, 20, 1...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",1.0
15,11.0,26.0,1.341340,15.0,2.0,25.0,1.433421,0.092081,"[15, 29, 16, 19, 20, 9, 32, 23, 11, 18, 25, 14...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",1.0
...,...,...,...,...,...,...,...,...,...,...,...
134238,49.0,104.0,1.309114,134239.0,7507.0,103.0,1.483535,0.174421,"[134239, 134181, 134189, 134197, 134207, 13418...","[7507, 7507, 7507, 7507, 7507, 7507, 7507, 750...",1.0
134251,33.0,104.0,1.350507,134252.0,7507.0,103.0,1.472909,0.122402,"[134252, 134209, 134272, 134253, 134249, 13423...","[7507, 7507, 7507, 7507, 7507, 7507, 7507, 750...",1.0
134260,40.0,104.0,1.317282,134261.0,7507.0,103.0,1.460080,0.142798,"[134261, 134204, 134268, 134214, 134222, 13421...","[7507, 7507, 7507, 7507, 7507, 7507, 7507, 750...",1.0
134267,40.0,104.0,1.367114,134268.0,7507.0,103.0,1.483535,0.116421,"[134268, 134195, 134238, 134274, 134204, 13419...","[7507, 7507, 7507, 7507, 7507, 7507, 7507, 750...",1.0


In [23]:
r = ResultContainer()
r.load('/tmp/testfeature/')

Load results and events from '/tmp/testfeature/'


In [27]:
cases = r.events[(r.events.topk_ap == 1.0) & (r.events.topk_purity != 1.0)]

In [34]:
sample = cases.sample(1)

In [31]:
r.events[(r.events.topk_ap <1.0)].ret_label_ids

8      [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 38, 38, 38, 38,...
12     [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 70, 2,...
16     [2, 2, 2, 2, 2, 2, 2, 2, 70, 2, 2, 38, 70, 2, ...
35     [7, 7, 7, 7, 109, 7, 7, 7, 7, 109, 7, 7, 7, 7,...
36     [7, 7, 7, 7, 109, 7, 109, 7, 7, 109, 109, 7, 7...
                             ...                        
903    [109, 109, 109, 7, 109, 7, 7, 7, 7, 7, 7, 109,...
904    [109, 109, 109, 109, 109, 7, 7, 7, 7, 7, 109, ...
905    [109, 109, 109, 109, 7, 7, 109, 109, 7, 109, 7...
928    [114, 114, 114, 114, 114, 106, 114, 106, 106, ...
936    [114, 114, 114, 114, 106, 114, 106, 106, 114, ...
Name: ret_label_ids, Length: 312, dtype: object

In [39]:
len(sample.ret_label_ids.values[0])

30

In [52]:
hit = sample.ret_label_ids.values[0] == sample.label_id.values

In [43]:
from feature_analyzer.metrics.ranking_metrics import RankingMetrics
ranker = RankingMetrics(15)

In [55]:
ranker.add_inputs(np.asarray([hit]))

In [56]:
ranker.average_precisions[0]

array([0.95878788])

In [57]:
hit

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False, False, False, False, False,
       False,  True, False,  True, False, False, False, False, False,
       False, False, False])

In [58]:
indices = np.where(hit)[0]